In [3]:
import cv2
import numpy as np

def dynamic_gamma_correction(img, clip_hist_percent=1):
    """动态伽马校正"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
    
    # 计算累积直方图
    accumulator = np.cumsum(hist)
    max_accumulator = accumulator[-1]
    
    # 查找高低阈值
    clip_hist = clip_hist_percent * max_accumulator / 100.0
    min_gray = np.where(accumulator >= clip_hist)[0][0]
    max_gray = np.where(accumulator >= max_accumulator - clip_hist)[0][0]
    
    # 动态计算伽马值
    current_contrast = max_gray - min_gray
    gamma = np.log(128) / np.log(current_contrast * 0.5) if current_contrast > 0 else 1.0
    gamma = np.clip(gamma, 0.3, 2.0)
    
    # 应用伽马校正
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def adaptive_glare_suppression(img, brightness_thresh=250, blur_kernel=(15,15)):
    """自适应强光抑制核心算法"""
    # 转换到YUV颜色空间
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    y_channel = yuv[:,:,0].astype(np.float32)
    
    # 步骤1：初步光源检测
    _, bright_mask = cv2.threshold(yuv[:,:,0], brightness_thresh, 220, cv2.THRESH_BINARY)
    bright_mask = cv2.morphologyEx(bright_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15)))
    
    # 步骤2：动态范围压缩
    y_compressed = np.where(y_channel > brightness_thresh, 
                           brightness_thresh + (y_channel - brightness_thresh)*0.3,
                           y_channel)
    
    # 步骤3：局部对比度增强
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    y_enhanced = clahe.apply(y_compressed.astype(np.uint8))
    
    # 步骤4：区域融合
    blend_mask = cv2.GaussianBlur(bright_mask.astype(np.float32)/255, (51,51), 0)
    y_final = y_channel*(1-blend_mask) + y_enhanced.astype(np.float32)*blend_mask
    
    # 合并通道并转换颜色空间
    yuv[:,:,0] = np.clip(y_final, 0, 255).astype(np.uint8)
    result = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    
    # 最终动态伽马校正
    return dynamic_gamma_correction(result)

# 使用示例（支持视频流处理）
if __name__ == "__main__":
    cap = cv2.VideoCapture("highway_n.avi")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        # 处理帧
        processed = adaptive_glare_suppression(frame)
        
        # 显示对比
        cv2.imshow("Original", cv2.resize(frame, (640,360)))
        cv2.imshow("Processed", cv2.resize(processed, (640,360)))
        
        if cv2.waitKey(1) & 0xFF == 27:
            break
    
    cap.release()
    cv2.destroyAllWindows()